In [1]:
import os
os.chdir('/home/piper/work')

import numpy as np
import megatron

In [2]:
# lazy
G = megatron.Graph(cache_dir='megatron/cache')
X = megatron.Input(G, 'X', (10,))
Y = megatron.Input(G, 'Y', (10,))

X1 = megatron.Transformation(megatron.transforms.TimeSeries(window_size=5))(X)
Y1 = megatron.Transformation(megatron.transforms.TimeSeries(window_size=5))(Y)
Z = megatron.Transformation(megatron.transforms.Add())(X1, Y1)

data = {'X': np.random.random((50, 10)),
        'Y': np.random.random((50, 10))}

result = G.run(Z, data, cache_result=False)
print(result.shape)

(45, 5, 10)


In [5]:
# eager
data = {'X': np.random.random((50, 10)),
        'Y': np.random.random((50, 10))}

G = megatron.Graph()
X = megatron.Input(G, 'X', (10,))(data['X'])
Y = megatron.Input(G, 'Y', (10,))(data['Y'])
X1 = megatron.Transformation(megatron.transforms.TimeSeries(window_size=5))(X)
Y1 = megatron.Transformation(megatron.transforms.TimeSeries(window_size=5))(Y)
Z = megatron.Transformation(megatron.transforms.Add())(X1, Y1)
print(X.output.shape)
print(Z.output.shape)

(50, 10)
(45, 5, 10)
